[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/diogoflim/Pesquisa-Operacional-III-A/blob/main/12_Intro_SimPy.ipynb)

## **Pesquisa Operacional III-A**

**Professor:**
- Diogo Ferreira de Lima Silva (TEP-UFF)


In [ ]:
!pip install simpy

In [ ]:
import simpy # usado para a modelagem
import random # gerador de números randômicos
import numpy as np # usado para calcular algumas métricas
import pandas as pd # usado para limpeza de dados e DataFrames 

In [ ]:
random.seed(2)

### Exemplo de processo que não alcança estado estável

Exemplo de processo M/M/2, com $\lambda = 1$ cliente/minuto e $\mu = 0,2$ clientes/minuto.

Como $\rho = \frac{\lambda}{s\times \mu} = \frac{1}{0,4} >1$, é esperado que o processo não estabilize.

A fila explodirá. 

Altere o código e calcule o tempo médio de espera dos clientes que finalizaram o serviço.

In [ ]:
def estabelecimento (environment, nome_cliente, servidor):
    #imprima na tela o tempo de chegada.
    print (f"{nome_cliente} chega ao estabelecimento em {environment.now}") 
    
    #guarde a informação na variável tempo_chegada.
    tempo_chegada = environment.now 
    
    # A atividade requer um servidor. Usamos o with para demonstrar isso. 
    with servidor.request() as req:
        # Espere até o servidor estar disponível e guarde o tempo.
        yield req 
        
        # Escreva na tela o tempo de início do serviço
        print (f"{nome_cliente} inicia o serviço em {environment.now}") 
        # guarde o tempo de início em uma variável chamada tempo_inicio
        tempo_inicio = environment.now

        # tempo de espera será o tempo_inicio - tempo_chegada 
        tempo_em_fila.append(tempo_inicio - tempo_chegada)

        # O tempo estimado segue uma exponencial trabalha com Mu=0,2
        yield environment.timeout(random.expovariate(0.2))  
        
        #Imprima na tela o tempo de saída do sistema
        print (f"{nome_cliente} finaliza o serviço e sai do estabelecimento em {environment.now}")
        tempo_saida = environment.now
        
        # O tempo no sistema será tempo_saida - tempo_chegada
        tempo_no_sistema.append(tempo_saida - tempo_chegada) 

def chegadas (environment):
    id = 1 #guarda o id do cliente 
    
    # Enquanto houver simulação:
    while True:
        # Passa um tempo até a próxima chegada, seguindo exponencial com média (1/lambda)
        yield environment.timeout(random.expovariate(1))
        
        # Um cliente chega no processo
        environment.process(estabelecimento (environment, 'Cliente %d' % id, servidor))
        
        # O próximo cliente terá id = id + 1
        id += 1



In [ ]:
tempo_em_fila = [] # uma lista vazia que receberá os tempos em fila
tempo_no_sistema = [] # uma lista vazia que receberá os tempos no sistema de filas

ambiente = simpy.Environment()
servidor = simpy.Resource(ambiente, capacity=2)
ambiente.process(chegadas(ambiente))
ambiente.run(until = 480)

In [ ]:
print (f"Número de clientes que iniciaram o atendimento : {len(tempo_em_fila)}")
print (f"O tempo de espera médio: {np.mean(tempo_em_fila).round(2)} minutos")
print (f"O tempo médio no sistema: {np.mean(tempo_no_sistema).round(2)} minutos")

Veja que o tempo de espera médio foi de mais de 2 horas.

Na verdade, só contabilizamos tempos de espera que conseguiram iniciar o atendimento.

Perceba que chegaram mais de 400 clientes e menos da metade deles foram atendidos.

Seria interessante termos uma forma de guardar a utilização dos recursos e o número de clientes em fila!

# Analisando Recursos

A análise de recursos não é tão simples quanto guardar os tempos dos eventos. Para realizar uma análise efetiva, precisamos acessar os recursos após cada mudança no sistema.

A boa notícia é que já há uma função pronta disponibilizada na documentação do SimPy, copiada na célula abaixo.

Nesse código, usaremos decoradores de função (wrapper) para modificar recursos do SimPy, de modo que armazenaremos algumas características  antes e após alguma no mesmo.

O código abaixo é semelhante ao disponível na documentação. Algumas modificações foram realizadas em termos usados e nos comentários.

- https://simpy.readthedocs.io/en/latest/topical_guides/monitoring.html

In [ ]:
from functools import partial, wraps

def modifica_recurso (recurso, pre=None, post=None):
    '''
    Modificaremos o recurso para que este chame o *pre* antes de cada operação 
    e o *post* após cada operação.
    *pre* e *post* fazem parte do wraps, que chamamos do pacote functools.
    Operações típicas de recursos do SimPy são: put, get, request e release.
    '''

    def get_decorador(func):
        
        # Cria um decorador para as funções internas dos recursos: put/get/request/release
        @wraps(func)
        def decorador(*args, **kwargs):
            # Chame o "pre" antes de aplicar a função
            if pre:
                pre(recurso)

            # aplique a função
            ret = func(*args, **kwargs)
            
            # Chame o "post" após aplicar a função
            if post:
                post(recurso)
            
            return ret #retorna o que a função retornaria
        return decorador # retorna a função modificada pelo decorador

    # Modifique a operação original pela modificada com o wrapper
    for nome in ['put', 'get', 'request', 'release']:
        
        # Se acontecer uma operação do tipo nome no recurso em questão, mude os atributos
        if hasattr(recurso, nome):
            setattr(recurso, nome, get_decorador(getattr(recurso, nome)))


# Armazenaremos o que queremos do recurso em questão em uma lista chamada dados
def monitoramento(dados_controle, recurso):
    item = (
            recurso._env.now,  # O tempo de simulação na ocorrência de uma operação
            recurso.count,  # Número de recursos do tipo em questão sendo utilizados
            len(recurso.queue),  # Número de trabalhos em fila para o recurso
    )   
    dados_controle.append(item)

In [ ]:
tempo_em_fila = [] # uma lista vazia que receberá os tempos em fila
tempo_no_sistema = [] # uma lista vazia que receberá os tempos no sistema de filas

ambiente2 = simpy.Environment()
servidor = simpy.Resource(ambiente2, capacity=2)

dados_controle=[]
monitoramento = partial(monitoramento, dados_controle)
modifica_recurso (servidor, post = monitoramento)


ambiente2.process(chegadas(ambiente2))
ambiente2.run(until = 480)

## Vejamos os dados que conseguimos!

In [ ]:
dados_controle

### Organizando os dados dataframe

In [ ]:
servidor_dados = pd.DataFrame(dados_controle, columns = ["Tempo", "n", "n_q"])
servidor_dados

### Podemos adicionar uma linha no topo para ser o estado inicial, quando nenhum cliente chegou:

In [ ]:
servidor_dados.loc[-1] = [0.0, 0, 0]  # adding a row
servidor_dados.index = servidor_dados.index + 1  # shifting index
servidor_dados.sort_index(inplace=True) 
servidor_dados

### Agora adicionarems uma linha no fim com o estado do sistema no momento 480.

In [ ]:
servidor_dados.loc[len(servidor_dados)] = [480, 2, 333]
servidor_dados

### Queremos o tempo em que observamos cada estado antes de um novo evento!

Podemos chegar a isso com "servidor_1.diff()" aplicado à coluna tempo

In [ ]:
servidor_dados["Intervalo"] = servidor_dados["Tempo"].diff().shift(-1)
servidor_dados

A última linha ficou com um NaN. Vamos excluí-la:


In [ ]:
servidor_dados = servidor_dados.dropna(axis = 0)
servidor_dados

## Calculando métricas interessantes

### Número médio de clientes em fila


Sabemos que a fila está explodindo. Vejamos a média de clientes em fila nesses 480 min simulados


In [ ]:
servidor_dados["Intervalo"] @ servidor_dados["n_q"] / servidor_dados["Intervalo"].sum()

### Uso dos servidores, lembrando que em nosso exemplo $s=2$

In [ ]:
servidor_dados["Intervalo"] @ servidor_dados["n"] / servidor_dados["Intervalo"].sum()

### Porcentagem do tempo que não temos ninguem no sistema

In [ ]:
servidor_ocioso = servidor_dados[servidor_dados["n"] == 0]
servidor_ocioso

In [ ]:
servidor_ocioso["Intervalo"].sum() / servidor_dados["Intervalo"].sum()